# Laboratorium 5 - rekomendacje grafowe

## Przygotowanie

 * dataset i potrzebne biblioteki są dokładnie takie same jak na poprzednim laboratorium
 * pobierz i wypakuj dataset: https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
   * więcej możesz poczytać tutaj: https://grouplens.org/datasets/movielens/
 * [opcjonalnie] Utwórz wirtualne środowisko
 `python3 -m venv ./recsyslab5`
 * zainstaluj potrzebne biblioteki:
 `pip install numpy pandas sklearn gensim==3.8.3`

## Część 1. - przygotowanie danych

In [45]:
# importujemy wszystkie potrzebne pakiety

import math
import random
import numpy as np
import pandas

from gensim.models import Word2Vec

from sklearn.model_selection import train_test_split, KFold

In [46]:
SCORE_THRESHOLD = 4.0 # recenzje z co najmniej taka ocena wezmiemy pod uwage
VECTOR_SIZE = 20 # jak dlugie powinny byc wektory osadzen wierzcholkow
NEIGHBOURS_WINDOW = 11 # tylu sasiadow wezmiemy pod uwage w algorytmie Word2Vec (symetrycznie i wliczajac biezacy element)
PATH_LENGTH = 30 # dlugosc pojedynczej losowej sciezki
PATHS_COUNT_PER_NODE = 20 # liczba losowych sciezek zaczynajacych sie w kazdym z wierzcholkow

In [47]:
# wczytujemy oceny uytkownikow

ratings = pandas.read_csv('ml-latest-small/ratings.csv').drop(columns=['timestamp'])
ratings = ratings.where(ratings['rating'] >= SCORE_THRESHOLD).dropna()
# rozszerzamy ID tak, by sie nie powtarzaly
ratings['userId'] = ratings['userId'].apply(lambda x: 'u_' + str(int(x)))
ratings['movieId'] = ratings['movieId'].apply(lambda x: 'm_' + str(int(x)))
ratings

,userId,movieId,rating
0,u_1,m_1,4.0
1,u_1,m_3,4.0
2,u_1,m_6,4.0
3,u_1,m_47,5.0
4,u_1,m_50,5.0
...,...,...,...
100830,u_610,m_166528,4.0
100831,u_610,m_166534,4.0
100832,u_610,m_168248,5.0
100833,u_610,m_168250,5.0


In [48]:
# wczytujemy gatunki filmow

movies = pandas.read_csv('ml-latest-small/movies.csv').drop(columns=['title'])
movies['movieId'] = movies['movieId'].apply(lambda x: 'm_' + str(int(x)))
movies['genres'] = movies['genres'].apply(lambda x: x.split('|'))
movies_to_genres = movies.explode('genres')
movies_to_genres['genres'] = movies_to_genres['genres'].apply(lambda x: 'g_' + x.lower())
movies_to_genres = movies_to_genres.rename(columns = {'genres': 'genre'})
movies_to_genres

,movieId,genre
0,m_1,g_adventure
0,m_1,g_animation
0,m_1,g_children
0,m_1,g_comedy
0,m_1,g_fantasy
...,...,...
9738,m_193583,g_fantasy
9739,m_193585,g_drama
9740,m_193587,g_action
9740,m_193587,g_animation


In [49]:
users = ratings['userId'].unique()
movies = ratings['movieId'].unique()
genres = movies_to_genres['genre'].unique()

In [60]:
movies_to_title = pandas.read_csv('ml-latest-small/movies.csv').drop(columns=['genres'])
movies_to_title['movieId'] = movies_to_title['movieId'].apply(lambda x: 'm_' + str(int(x)))
movies_to_title = movies_to_title.set_index('movieId')
movies_to_title

,title
movieId,
m_1,Toy Story (1995)
m_2,Jumanji (1995)
m_3,Grumpier Old Men (1995)
m_4,Waiting to Exhale (1995)
m_5,Father of the Bride Part II (1995)
...,...
m_193581,Black Butler: Book of the Atlantic (2017)
m_193583,No Game No Life: Zero (2017)
m_193585,Flint (2017)


## Część 2. - spacer po grafie

In [20]:
# generujemy losowe sciezki w grafie
#   krawedzie reprezentowane sa w dwoch macierzach - ratings i movies
#   w wersji podstawowej wszystkie krawedzie traktujemy jako niewazone i nieskierowane
#   mozliwe ulepszenia:
#    - rozwazenie krawedzi skierowanych
#    - uwzglednienie wag krawedzi (ocen uzytkownikow)
#    - jakas forma normalizacji - obnizenia wag wierzcholkow o wysokich stopniach
#    - Node2Vec - parametry P i Q
# wynikiem powinna byc lista list - kazda z tych list zawiera kolejne ID wierzcholkow na sciezce    
from functools import reduce

nodes = users.tolist()
nodes.extend(movies.tolist())
nodes.extend(genres.tolist())

np_ratings = ratings.to_numpy()
np_movies_to_genres = movies_to_genres.to_numpy()
    
def get_edges():
    b = np.hstack((np_movies_to_genres, np.zeros((np_movies_to_genres.shape[0], 1), dtype=np_movies_to_genres.dtype)))
    b[:, [0,1]] = b[:, [1,0]]
    edges = np.vstack((b, np_ratings))
    edges_swapped = edges.copy()
    edges_swapped[:, [0,1]] = edges_swapped[:, [1,0]]
    edges = np.vstack((edges, edges_swapped))
    return edges
    
edges = get_edges()
edges_from_node = {}

def next_node(node):
    if not node in edges_from_node:
        edges_from_node[node] = edges[np.where(edges[:,1] == node)[0]][:, 0]
    edge = np.random.choice(edges_from_node[node])
    return edge    
    
def get_path(node):
    return [node] + [node := next_node(node) for _ in range(PATH_LENGTH)]

def get_paths(node):
    return [get_path(node) for _ in range(PATHS_COUNT_PER_NODE)]

In [21]:
import multiprocess as mp

paths = []
with mp.Pool(processes=8) as p:
    l = p.map(get_paths, nodes)
    paths = reduce(lambda x, y: x+y, l)

## Część 3. - obliczenie osadzeń

In [25]:
# trenujemy model
#   zauwaz, ze wszystkie trzy rodzaje wierzcholkow beda reprezentowane tak samo, w tej samej przestrzeni

model = Word2Vec(sentences=paths, size=VECTOR_SIZE, window=NEIGHBOURS_WINDOW, min_count=1, workers=4)
embeddings = model.wv

## Część 4. - rekomendacje i zastosowania

In [26]:
PULP_FICTION = 'm_296'
TOY_STORY = 'm_1'
PLANET_OF_THE_APES = 'm_2529'

In [29]:
embeddings[PULP_FICTION]

array([ 3.9171317 ,  0.31117243,  4.140499  ,  5.406548  , -9.999408  ,
       -0.7102429 , -5.3674626 ,  3.0218813 ,  2.2469797 , -3.0358868 ,
        3.4642434 , -5.2033367 , -1.9366151 , 10.775511  , -3.0510905 ,
        3.274083  ,  4.9235272 , -4.4009027 , -8.200265  ,  3.2142394 ],
      dtype=float32)

In [199]:
# wyszukajmy K najpodobniejszych filmów do danego
# porownaj wyniki dla odleglosci euklidesowej i cosinuswej, np. na trzech powyzszych filmach

def euclidian_distance(i, j):
    return np.sum(np.power((i-j), 2))

def cosine_distance(i, j):
    return np.dot(i, j)/np.linalg.norm(i)/np.linalg.norm(j)

def k_most_similar_movies(movie_id, K, embeddings, distance_fun):
    distances = [(mid, distance_fun(embeddings[movie_id], embeddings[mid])) for mid in movies] 
    k_most_similar = list(map(lambda x: x[0], sorted(distances, key=lambda x: x[1])[:K]))
    return k_most_similar

In [111]:
def print_titles(k_most):
    print('\n'.join([f'{mid}: {movies_to_title.loc[mid].title} ' for mid in k_most]))

In [ ]:
movies_to_genres = movies_to_genres.set_index('movieId')

In [200]:
def print_genres(movies):
    for mid in movies:
        gen = movies_to_genres.loc[mid]['genre']
        print(mid, ':', str([gen]) if isinstance(gen, str) else str(gen.to_list()))

In [201]:
k = 5
method_name = {
    cosine_distance: 'cosine_distance',
    euclidian_distance: 'euclidian_distance'
}
for movie in [PULP_FICTION, TOY_STORY,PLANET_OF_THE_APES]:
    for method in [euclidian_distance, cosine_distance]:
        k_most = k_most_similar_movies(movie, k, embeddings, method)
        print(f"Movie: {movies_to_title.loc[movie].title}, method: {method_name[method]}")
        print_titles(k_most)

Movie: Pulp Fiction (1994), method: euclidian_distance
m_296: Pulp Fiction (1994) 
m_593: Silence of the Lambs, The (1991) 
m_47: Seven (a.k.a. Se7en) (1995) 
m_50: Usual Suspects, The (1995) 
m_318: Shawshank Redemption, The (1994) 
Movie: Pulp Fiction (1994), method: cosine_distance
m_73822: Meantime (1984) 
m_180045: Molly's Game (2017) 
m_118862: Closer to the Moon (2013) 
m_795: Somebody to Love (1994) 
m_164707: Go Figure (2005) 
Movie: Toy Story (1995), method: euclidian_distance
m_1: Toy Story (1995) 
m_1073: Willy Wonka & the Chocolate Factory (1971) 
m_364: Lion King, The (1994) 
m_588: Aladdin (1992) 
m_260: Star Wars: Episode IV - A New Hope (1977) 
Movie: Toy Story (1995), method: cosine_distance
m_116411: Tangerines (2013) 
m_6376: Respiro (2002) 
m_4883: Town is Quiet, The (Ville est tranquille, La) (2000) 
m_183199: Quest (2017) 
m_107408: Only Old Men Are Going to Battle (V boy idut odni stariki) (1973) 
Movie: Planet of the Apes (1968), method: euclidian_distance
m_25

In [202]:
# wyszukajmy k filmow najblizszych uzytkownikowi
# wykorzystaj funkcje z poprzedniej komorki

def k_best_movies_for_user(user_id, K, embeddings, distance_fun):
    return k_most_similar_movies(user_id, K, embeddings, distance_fun)

In [203]:
def k_most_similar_from_favourite_genre(user_id, K, embeddings, distance_fun):
    genre_distances = [(gid, distance_fun(embeddings[user_id], embeddings[gid])) for gid in genres] 
    genre = sorted(genre_distances, key=lambda x: x[1])[0][0]
    print('Favourite genre ', genre)
    return k_most_similar_movies(genre, K, embeddings, distance_fun)

In [204]:
user = random.choice(users)
user

'u_505'

In [205]:
m = k_most_similar_from_favourite_genre(user, 5, embeddings, euclidian_distance)
print_titles(m)
print()
print_genres(m)

Favourite genre  g_horror
m_6731: Day of the Dead (1985) 
m_2867: Fright Night (1985) 
m_103688: Conjuring, The (2013) 
m_968: Night of the Living Dead (1968) 
m_3917: Hellraiser (1987) 

m_6731 : ['g_horror', 'g_sci-fi', 'g_thriller']
m_2867 : ['g_comedy', 'g_horror', 'g_thriller']
m_103688 : ['g_horror', 'g_thriller']
m_968 : ['g_horror', 'g_sci-fi', 'g_thriller']
m_3917 : ['g_horror']


In [206]:
m = k_most_similar_from_favourite_genre(user, 5, embeddings, cosine_distance)
print_titles(m)
print()
print_genres(m)

Favourite genre  g_romance
m_4105: Evil Dead, The (1981) 
m_53972: Live Free or Die Hard (2007) 
m_2841: Stir of Echoes (1999) 
m_57368: Cloverfield (2008) 
m_98961: Zero Dark Thirty (2012) 

m_4105 : ['g_fantasy', 'g_horror', 'g_thriller']
m_53972 : ['g_action', 'g_adventure', 'g_crime', 'g_thriller']
m_2841 : ['g_horror', 'g_mystery', 'g_thriller']
m_57368 : ['g_action', 'g_mystery', 'g_sci-fi', 'g_thriller']
m_98961 : ['g_action', 'g_drama', 'g_thriller']


In [207]:
# sprobujmy czegos bardziej skomplikowanego
#   znajdz ulubiony gatunek filmowy uzytkownika
#   a nastepnie zaproponuj K filmow z tego gatunku - ale nie tych najblizszych uzytkownikowi
#   (zaproponuj, w jaki sposob dobrac filmy interesujace, ale nie z najblizszego otoczenia)

def k_from_favourite_genre(user_id, K, embeddings, distance_fun):
    genre_distances = [(gid, distance_fun(embeddings[user_id], embeddings[gid])) for gid in genres] 
    genre = sorted(genre_distances, key=lambda x: x[1])[0][0]
    print('Favourite genre ', genre)
    distances = [(mid, distance_fun(embeddings[genre], embeddings[mid])) for mid in movies] 
    distanced = list(map(lambda x: x[0], sorted(distances, key=lambda x: x[1])[len(distances)//10:len(distances)//10*5]))
    return random.choices(distanced, k=k)

In [208]:
m = k_from_favourite_genre(user, 5, embeddings, euclidian_distance)
print_titles(m)
print()
print_genres(m)

Favourite genre  g_horror
m_105746: UnHung Hero (2013) 
m_8813: We Don't Live Here Anymore (2004) 
m_5294: Frailty (2001) 
m_1652: Year of the Horse (1997) 
m_128366: Patton Oswalt: Tragedy Plus Comedy Equals Time (2014) 

m_105746 : ['g_documentary']
m_8813 : ['g_drama']
m_5294 : ['g_crime', 'g_drama', 'g_thriller']
m_1652 : ['g_documentary']
m_128366 : ['g_comedy']


In [209]:
m = k_from_favourite_genre(user, 5, embeddings, cosine_distance)
print_titles(m)
print()
print_genres(m)

Favourite genre  g_romance
m_93766: Wrath of the Titans (2012) 
m_1230: Annie Hall (1977) 
m_2078: Jungle Book, The (1967) 
m_61024: Pineapple Express (2008) 
m_163639: DC Super Hero Girls: Hero of the Year (2016) 

m_93766 : ['g_action', 'g_adventure', 'g_fantasy', 'g_imax']
m_1230 : ['g_comedy', 'g_romance']
m_2078 : ['g_animation', 'g_children', 'g_comedy', 'g_musical']
m_61024 : ['g_action', 'g_comedy', 'g_crime']
m_163639 : ['g_animation']


In [210]:
# Na koniec najbardziej skomplikowany algorytm - odpowiednik "Radia utworu" w Spotify.
#   Zaczynamy od jednego filmu, a nastepnie wyznaczamy kolejne, wedrujac po przestrzeni, w ktorej wszystkie elementy sa osadzone.
#   Zaproponuj, jak zdefiniowac podzbior filmow, z ktorych bedziemy wybierac (np. filmy odlegle o min. a i max. b od danego)
#   oraz jak generowac kolejny skok (tak, zeby seria rekomendacji nie byla zbyt monotonna, ale rownoczesnie zgodna z gustem uzytkownika)

def get_playlist(start_movie_id, user_id, K, embeddings):
    # ...
    return playlist